In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
from dotenv import load_dotenv, find_dotenv

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)


_ = load_dotenv(find_dotenv())

# print(os.environ.get('OPENAI_API_KEY'))

In [3]:
import os
os.listdir("../../../HF_models")
os.listdir("../")

['testcases',
 'langgraph.json',
 'notebooks',
 'readme.md',
 '=0.9.2',
 '.gradio',
 '.env',
 '.langgraph_api',
 'src',
 'llm_app.py',
 'app.log',
 'audio_samples',
 'asset',
 'app_config.py',
 '__pycache__',
 'docs',
 'langgraph copy.json',
 '__init__.py']

In [ ]:
from transformers import pipeline, AutoModelForSpeechSeq2Seq, AutoProcessor
from transformers import BitsAndBytesConfig
import torch
# Load model components separately
torch_dtype = torch.float16
# MODEL_NAME = "../../../HF_models/whisper-large-v3-turbo"
MODEL_NAME ="openai/whisper-large-v3-turbo"

quantization_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.float16
            )

processor = AutoProcessor.from_pretrained(MODEL_NAME)
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch_dtype, 
    low_cpu_mem_usage=True,
    quantization_config=quantization_config
    )

# Create pipeline
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor
)

# Use the proper parameter structure
audio_path = "../audio_samples/20250210-181617_8783_27728204887-all.mp3"
result = pipe(
    audio_path,
    decoder_kwargs={"language": "en"},
    generate_kwargs={"task": "transcribe"},
    return_timestamps=True
    
)

print(result["text"])

Device set to use cuda:0


 Good day, how are you? Good day. Good, my name is Priti, calling from Card Track Accounts Department. Can I please speak to Phineas Mthongo? Yes, you must begin. Can that call to be recorded for quality and security purposes? Can you please confirm my ID number, sir? 740-225-385-03. Thank you very much. Sir, there is an accolge says because of the outstanding amount of 288,30 cents. Sorry, 288,30, yes, and 30 cents. And we need an immediate payment to be done today. Do you maybe have enough funds so that you can do the deduction? Come on, sir. Sorry? What are you saying? The account is behind with an amount of 288,30, 30 cents, sir. Yes. Yes. We need an immediate payment to be done today. Okay. Yes. Do you have enough funds that you can do the deduction? No fund. Sorry? No money. When can you be able to make the payment? Next month. next month on which day? on the fourth on the fourth so a double deduction? yeah on the fourth no problem I'll make a debit order then for the fourth and 

In [ ]:
# MODEL_NAME = "../../../HF_models/whisper-large-v3-turbo"
# # model.save_pretrained(MODEL_NAME)
# processor.save_pretrained(MODEL_NAME)

[]

### Whisper

In [3]:
import os
os.listdir("../audio_samples")

['20250218-172808_8562_27727903218-all.mp3',
 '20250218-172457_8788_27832667859-all.mp3',
 '20250219-093730_8688_27786753396-all.mp3',
 '20250210-181617_8783_27728204887-all.mp3',
 '20250220-083952_8689_27648301967-all.mp3']

In [10]:
from src.STT import create_stt_model
from app_config import CONFIG
print(CONFIG['stt'])
# Configure the model
config1 = CONFIG['stt'].copy()
config1["model_name"] = "openai/whisper-large-v3-turbo"

config2 = CONFIG['stt'].copy()
config2["model_name"] = "nvidia/parakeet-tdt-0.6b-v2"

# Create the model
model1 = create_stt_model(config1)
model2 = create_stt_model(config2)


2025-05-22 11:25:05,893 - WARNING - Unsupported STT model: openai/whisper-large-v3-turbo. Falling back to 'whisper-large-v3-turbo'. Supported models: whisper-large-v3-turbo, nvidia/parakeet-tdt-0.6b-v2
2025-05-22 11:25:05,894 - INFO - Creating whisper-large-v3-turbo STT model with config: {'checkpoint': 'whisper-large-v3-turbo', 'model_folder_path': '/home/ct-admin/Documents/Langgraph/HF_models/', 'batch_size': 8, 'cuda_device_id': 1, 'chunk_length_s': 30, 'compute_type': 'float16', 'beam_size': 3, 'show_logs': True}
2025-05-22 11:25:05,895 - INFO - Loading STT model from /home/ct-admin/Documents/Langgraph/HF_models/whisper-large-v3-turbo on cuda:1


{'model_name': 'whisper-large-v3-turbo', 'whisper-large-v3-turbo': {'checkpoint': 'whisper-large-v3-turbo', 'model_folder_path': '/home/ct-admin/Documents/Langgraph/HF_models/', 'batch_size': 8, 'cuda_device_id': 1, 'chunk_length_s': 30, 'compute_type': 'float16', 'beam_size': 3}, 'nvidia/parakeet-tdt-0.6b-v2': {'timestamp_prediction': True, 'decoding_type': 'tdt'}}
Device set to use cuda:1


Device set to use cuda:1
2025-05-22 11:25:06,526 - INFO - STT model whisper-large-v3-turbo loaded successfully
2025-05-22 11:25:06,529 - INFO - Creating nvidia/parakeet-tdt-0.6b-v2 STT model with config: {'timestamp_prediction': True, 'decoding_type': 'tdt', 'show_logs': True}
2025-05-22 11:25:06,530 - INFO - Device set to use cuda:0
2025-05-22 11:25:06,530 - INFO - Using checkpoint name directly: nvidia/parakeet-tdt-0.6b-v2


[NeMo I 2025-05-22 11:25:07 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2025-05-22 11:25:07 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    use_lhotse: true
    skip_missing_manifest_entries: true
    input_cfg: null
    tarred_audio_filepaths: null
    manifest_filepath: null
    sample_rate: 16000
    shuffle: true
    num_workers: 2
    pin_memory: true
    max_duration: 40.0
    min_duration: 0.1
    text_field: answer
    batch_duration: null
    use_bucketing: true
    bucket_duration_bins: null
    bucket_batch_size: null
    num_buckets: 30
    bucket_buffer_size: 20000
    shuffle_buffer_size: 10000
    
[NeMo W 2025-05-22 11:25:07 nemo_logging:405] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config :

[NeMo I 2025-05-22 11:25:07 nemo_logging:393] PADDING: 0
[NeMo I 2025-05-22 11:25:09 nemo_logging:393] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}
[NeMo I 2025-05-22 11:25:09 nemo_logging:393] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}
[NeMo I 2025-05-22 11:25:09 nemo_logging:393] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}
[NeMo I 2025-05-22 11:25:10 nemo_logging:393] Model EncDecRNNTBPEModel was successfully restored from /home/ct-admin/.cache/huggingface/hub/models--nvidia--parakeet-tdt-0.6b-v2/snapshots/50aec6a056e85b9f95b612df08a2bddc55b58714/parakeet-tdt-0.6b-v2.nemo.


2025-05-22 11:25:10,300 - INFO - NVIDIA Parakeet-TDT model loaded successfully


In [11]:
# Transcribe an audio file
audio_path = "../audio_samples/20250210-181617_8783_27728204887-all.mp3"

result1 = model1.transcribe(audio_path)
print(f"Transcription1: {result1['text']}")

result2 = model2.transcribe(audio_path)
print("="*50)
print(f"Transcription2: {result2['text']}")

# # Print timestamps if available
# if "timestamps" in result:
#     for word, start, end in result["timestamps"]:
#         print(f"'{word}': {start:.2f}s - {end:.2f}s")

`generation_config` default values have been modified to match model-specific defaults: {'num_beams': 3}. If this is not desired, please set these values explicitly.


Transcription1:  Good day, how are you? Good day. Good, my name is Priti, calling from Card Track Accounts Department. Can I please speak to Phineas Mthongo? Yes, you must begin. Can you please confirm your ID number, sir? Thank you very much. So that is in our calls, you alcohol justice because of the outstanding amount of 288 ran in 30 cents and we need an immediate payment to be done today. Do you maybe have enough fun so that you can do the deduction? Come on back. Sorry? What are you saying? Yes. We need an immediate payment to be done today. Okay. Yes. Do you have enough funds so that you can do the deduction? No fund. Sorry? No money. When can you be able to make the payment? Next month. Next month on which day? On the 4th. On the 4th. On the 4th. So a double deduction. Yeah, on the 4th. No problem. I'll make it a bit older than for the 4th and then you're going to get an SMS from the bank asking you to authorize the mandate. Listen so that you authorize it and also there's goin

Transcribing: 100%|██████████| 1/1 [00:00<00:00,  1.68it/s]

Transcription2: Good day, how are you? Good day. I'm good. My name is Prithi Colling from Catrak Accounts Department. Can I please speak to Vineyard Sumtongo? Can not that call to be recorded for quality and security purposes? Can you please confirm your IT number, sir? 742263843. Thank you very much. So there is an accordion because of the outstanding amount of 288 run in Delicent and 288 run yes and this is and we need an immediate payment to be done today. Do you maybe have enough funds so that you can do the deduction? Come on. Sorry? What are you saying? The account is behind with an amount of two hundred and eighty eight run in this end sir. Yes. Yes, we need an immediate payment to be done today. Okay. Yes. Do you have enough funds so that you can do the deduction? No found. Sorry? No. No money. When can you be able to make the payment? Next month. Next month on which day? On the fourth. On the fourth. On the fourth, so a double deduction. Yeah, on the fourth. No problem. I'll m

### NVIDIA Parakeet-TDT

In [2]:
import nemo.collections.asr as nemo_asr
asr_model = nemo_asr.models.ASRModel.from_pretrained(model_name="nvidia/parakeet-tdt-0.6b-v2")

audio_path = "../audio_samples/20250210-181617_8783_27728204887-all.mp3"

output = asr_model.transcribe(audio_path)
print(output[0].text)

/home/ct-admin/anaconda3/envs/llm_cv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[NeMo I 2025-05-06 13:28:43 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2025-05-06 13:28:43 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    use_lhotse: true
    skip_missing_manifest_entries: true
    input_cfg: null
    tarred_audio_filepaths: null
    manifest_filepath: null
    sample_rate: 16000
    shuffle: true
    num_workers: 2
    pin_memory: true
    max_duration: 40.0
    min_duration: 0.1
    text_field: answer
    batch_duration: null
    use_bucketing: true
    bucket_duration_bins: null
    bucket_batch_size: null
    num_buckets: 30
    bucket_buffer_size: 20000
    shuffle_buffer_size: 10000
    
[NeMo W 2025-05-06 13:28:43 nemo_logging:405] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config :

[NeMo I 2025-05-06 13:28:43 nemo_logging:393] PADDING: 0
[NeMo I 2025-05-06 13:28:45 nemo_logging:393] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}
[NeMo I 2025-05-06 13:28:45 nemo_logging:393] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}
[NeMo I 2025-05-06 13:28:45 nemo_logging:393] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}
[NeMo I 2025-05-06 13:28:46 nemo_logging:393] Model EncDecRNNTBPEModel was successfully restored from /home/ct-admin/.cache/huggingface/hub/models--nvidia--parakeet-tdt-0.6b-v2/snapshots/50aec6a056e85b9f95b612df08a2bddc55b58714/parakeet-tdt-0.6b-v2.nemo.


Transcribing: 100%|██████████| 1/1 [00:01<00:00,  1.13s/it]

Good day, how are you? Good day. I'm good. My name is Prithi Colling from Catrak Accounts Department. Can I please speak to Vineyard Sumtongo? Can not that call to be recorded for quality and security purposes? Can you please confirm your IT number, sir? 742263843. Thank you very much. So there is an accordion because of the outstanding amount of 288 run in Delicent and 288 run yes and this is and we need an immediate payment to be done today. Do you maybe have enough funds so that you can do the deduction? Come on. Sorry? What are you saying? The account is behind with an amount of two hundred and eighty eight run in this end sir. Yes. Yes, we need an immediate payment to be done today. Okay. Yes. Do you have enough funds so that you can do the deduction? No found. Sorry? No. No money. When can you be able to make the payment? Next month. Next month on which day? On the fourth. On the fourth. On the fourth, so a double deduction. Yeah, on the fourth. No problem. I'll make it a bit old

In [15]:
audio_path = "../audio_samples/20250210-181617_8783_27728204887-all.mp3"

output = asr_model.transcribe([audio_path], timestamps=True)
# by default, timestamps are enabled for char, word and segment level
word_timestamps = output[0].timestamp['word'] # word level timestamps for first sample
segment_timestamps = output[0].timestamp['segment'] # segment level timestamps
char_timestamps = output[0].timestamp['char'] # char level timestamps

print("="*100)
print(output[0].text)
print("="*100)


for stamp in segment_timestamps:
    print(f"{stamp['start']:.02f}s - {stamp['end']:.02f}s : {stamp['segment']}")


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing: 100%|██████████| 1/1 [00:00<00:00,  1.83it/s]

Good day, how are you? Good day. I'm good. My name is Prithi Colling from Catrak Accounts Department. Can I please speak to Vineyard Sumtongo? Can not that call to be recorded for quality and security purposes? Can you please confirm your IT number, sir? 742263843. Thank you very much. So there is an accordion because of the outstanding amount of 288 run in Delicent and 288 run yes and this is and we need an immediate payment to be done today. Do you maybe have enough funds so that you can do the deduction? Come on. Sorry? What are you saying? The account is behind with an amount of two hundred and eighty eight run in this end sir. Yes. Yes, we need an immediate payment to be done today. Okay. Yes. Do you have enough funds so that you can do the deduction? No found. Sorry? No. No money. When can you be able to make the payment? Next month. Next month on which day? On the fourth. On the fourth. On the fourth, so a double deduction. Yeah, on the fourth. No problem. I'll make it a bit old

In [7]:
from src.STT import create_stt_model

config = {
    "model_name": "nvidia/parakeet-tdt-0.6b-v2",
    "show_logs": False,
}

# Create the model
model = create_stt_model(config)

# Transcribe an audio file
audio_path = "../audio_samples/20250210-181617_8783_27728204887-all.mp3"


result = model.transcribe(audio_path)


# Print the result
print(f"Transcription: {result['text']}")

# # # Print timestamps if available
# # if "timestamps" in result:
# #     for word, start, end in result["timestamps"]:
# #         print(f"'{word}': {start:.2f}s - {end:.2f}s")

[NeMo W 2025-05-22 11:06:08 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    use_lhotse: true
    skip_missing_manifest_entries: true
    input_cfg: null
    tarred_audio_filepaths: null
    manifest_filepath: null
    sample_rate: 16000
    shuffle: true
    num_workers: 2
    pin_memory: true
    max_duration: 40.0
    min_duration: 0.1
    text_field: answer
    batch_duration: null
    use_bucketing: true
    bucket_duration_bins: null
    bucket_batch_size: null
    num_buckets: 30
    bucket_buffer_size: 20000
    shuffle_buffer_size: 10000
    
[NeMo W 2025-05-22 11:06:08 nemo_logging:405] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config :

Transcription: Good day, how are you? Good day. I'm good. My name is Prithi Colling from Catrak Accounts Department. Can I please speak to Vineyard Sumtongo? Can not that call to be recorded for quality and security purposes? Can you please confirm your IT number, sir? 742263843. Thank you very much. So there is an accordion because of the outstanding amount of 288 run in Delicent and 288 run yes and this is and we need an immediate payment to be done today. Do you maybe have enough funds so that you can do the deduction? Come on. Sorry? What are you saying? The account is behind with an amount of two hundred and eighty eight run in this end sir. Yes. Yes, we need an immediate payment to be done today. Okay. Yes. Do you have enough funds so that you can do the deduction? No found. Sorry? No. No money. When can you be able to make the payment? Next month. Next month on which day? On the fourth. On the fourth. On the fourth, so a double deduction. Yeah, on the fourth. No problem. I'll ma

In [9]:
result = model.transcribe(audio_path)
# Print the result
print(f"Transcription: {result['text']}")

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing: 100%|██████████| 1/1 [00:00<00:00,  1.94it/s]

Transcription: Good day, how are you? Good day. I'm good. My name is Prithi Colling from Catrak Accounts Department. Can I please speak to Vineyard Sumtongo? Can not that call to be recorded for quality and security purposes? Can you please confirm your IT number, sir? 742263843. Thank you very much. So there is an accordion because of the outstanding amount of 288 run in Delicent and 288 run yes and this is and we need an immediate payment to be done today. Do you maybe have enough funds so that you can do the deduction? Come on. Sorry? What are you saying? The account is behind with an amount of two hundred and eighty eight run in this end sir. Yes. Yes, we need an immediate payment to be done today. Okay. Yes. Do you have enough funds so that you can do the deduction? No found. Sorry? No. No money. When can you be able to make the payment? Next month. Next month on which day? On the fourth. On the fourth. On the fourth, so a double deduction. Yeah, on the fourth. No problem. I'll ma